# QRT Challenge Data 2021

## Summary

This exploratory notebook is a good starting point to help you make your first steps in the challenge.

We remind that the aim of the 2021 QRT Challenge Data is to determine the link between two types of assets: liquids and illiquids. We provide returns of 100 illiquid assets and the aim is to predict, for the same day, the sign of the return of 100 liquid assets.

In the following, we propose a very simple approach that determines for each liquid asset, the illiquid asset with maximum correlation. Thus we measures the $\beta$ (see definition [here](https://www.investopedia.com/terms/b/beta.asp)) between these assets which will be used for prediction.

This notebook is very straightforward, but if you have any question or comment, please ask it in the [forum](https://challengedata.qube-rt.com/).

In [2]:
import numpy as np
import pandas as pd

## Loading the data

In [3]:
X_train = pd.read_csv('./X_train.csv', index_col=0)
Y_train = pd.read_csv('./y_train.csv', index_col=0)
X_test = pd.read_csv('./X_test.csv', index_col=0)
X_train.head()

,ID_DAY,RET_216,RET_238,RET_45,RET_295,RET_230,RET_120,RET_188,RET_260,RET_15,...,RET_122,RET_194,RET_72,RET_293,RET_281,RET_193,RET_95,RET_162,RET_297,ID_TARGET
ID,,,,,,,,,,,,,,,,,,,,,
0,3316,0.004024,0.009237,0.004967,NaN,0.01704,0.013885,0.041885,0.015207,-0.003143,...,0.007596,0.01501,0.014733,-0.000476,0.006539,-0.010233,0.001251,-0.003102,-0.094847,139
1,3316,0.004024,0.009237,0.004967,NaN,0.01704,0.013885,0.041885,0.015207,-0.003143,...,0.007596,0.01501,0.014733,-0.000476,0.006539,-0.010233,0.001251,-0.003102,-0.094847,129
2,3316,0.004024,0.009237,0.004967,NaN,0.01704,0.013885,0.041885,0.015207,-0.003143,...,0.007596,0.01501,0.014733,-0.000476,0.006539,-0.010233,0.001251,-0.003102,-0.094847,136
3,3316,0.004024,0.009237,0.004967,NaN,0.01704,0.013885,0.041885,0.015207,-0.003143,...,0.007596,0.01501,0.014733,-0.000476,0.006539,-0.010233,0.001251,-0.003102,-0.094847,161
4,3316,0.004024,0.009237,0.004967,NaN,0.01704,0.013885,0.041885,0.015207,-0.003143,...,0.007596,0.01501,0.014733,-0.000476,0.006539,-0.010233,0.001251,-0.003102,-0.094847,217


## Reshaping the data

We transform the data so that each line corresponds to a specific day

In [4]:
idx_ret_features = np.where(X_train.columns.str.contains('RET'))[0]
init_ret_features = X_train.columns[idx_ret_features]
target_ret_features = 'RET_' + X_train['ID_TARGET'].map(str).unique()
returns = {}
for day in X_train.ID_DAY.unique():
    if day < 3800:
        u = X_train.loc[X_train.ID_DAY == day]
        a = u.iloc[0, idx_ret_features]
        b = Y_train[X_train.ID_DAY == day]['RET_TARGET']
        b.index = 'RET_' + u.ID_TARGET.map(str)
        returns[day] = pd.concat([a, b])
returns = pd.DataFrame(returns).T.astype(float)

returns_test = {}
for day in X_train.ID_DAY.unique():
    if day >= 3800:
        u = X_train.loc[X_train.ID_DAY == day]
        returns_test[day] = u.iloc[0, idx_ret_features]
returns_test = pd.DataFrame(returns_test).T.astype(float)
returns_test.head()

,RET_216,RET_238,RET_45,RET_295,RET_230,RET_120,RET_188,RET_260,RET_15,RET_150,...,RET_108,RET_122,RET_194,RET_72,RET_293,RET_281,RET_193,RET_95,RET_162,RET_297
3827,-0.004009,-0.005548,-0.021422,-0.030218,-0.005639,0.003338,-0.013247,0.005032,-0.005021,-0.021443,...,-0.004840,-0.008291,0.005553,0.001121,-0.006467,-0.001556,-0.001870,-0.009128,0.013800,-0.023336
3834,0.007707,0.008130,-0.004910,0.008931,-0.000735,-0.012948,0.000378,0.015277,-0.006204,0.026180,...,0.013966,0.013328,0.009513,0.003565,0.014544,0.036845,0.003394,0.003766,-0.022144,0.039878
3883,0.000521,0.002241,0.000343,-0.005655,0.036810,-0.058639,0.029246,0.016874,0.008989,-0.000141,...,-0.006882,0.004612,-0.014554,0.022569,0.005838,0.031823,0.043111,-0.004913,-0.008605,0.008947
3866,0.010061,-0.009760,-0.009352,-0.000984,-0.028529,0.062228,0.009471,-0.007391,-0.019664,0.005027,...,0.034180,-0.010298,0.003212,-0.015127,0.002810,0.003613,-0.054967,0.038480,-0.009314,-0.006411
3842,0.031291,-0.057148,-0.046133,-0.029021,-0.006266,-0.037852,-0.034172,NaN,-0.024918,0.015557,...,-0.012380,NaN,0.010108,-0.009225,-0.054563,0.001857,0.015830,-0.043131,0.015232,-0.037736


## Beta computation

We compute the $\beta$ between all assets. This matrix will determine the linear link between all assets.

This step is not necessary and could be done in the next step, but it is a good way to introduce the use of a matrix shrinkage, greatly used in finance when dealing with noisy data. See [here](https://scikit-learn.org/stable/auto_examples/covariance/plot_covariance_estimation.html) for more information.

In [5]:
from sklearn.covariance import oas

test = returns.cov().loc[init_ret_features, target_ret_features]

features = returns.columns
cov = pd.DataFrame(oas(returns.fillna(0))[0], index=features, columns=features)

beta = cov / np.diag(cov)

## Determine the pairs and beta coefficients

For each target asset (liquid assets), we determine the illiquid asset that has maximum correlation and we save the id and the associated beta coefficient.

In [6]:
proj_matrix = beta.T.loc[init_ret_features, target_ret_features]
corr = returns.corr().loc[init_ret_features, target_ret_features]

#print(corr)

coeffs = {}
for id_target in target_ret_features:
    x = proj_matrix[id_target]
    c = corr[id_target]
    id_init_max = c.abs().idxmax()
    j = id_target.replace('RET_', '')
    coeffs[j] = (id_init_max,  x[id_init_max])

## Prediction on test data

We thus simply make the predictions on the test data set using the pairs we saved and the beta.

If there is missing values, we replace them with the mean.

In [ ]:
X_t = X_train.sort_values(by=['ID_DAY'])
pred = {}
for idx, row in X_train.iterrows():
    if idx == 258791:
#     if row['ID_DAY'] >= 3900:
        print(idx)
        ret = returns.loc[:, init_ret_features].sort_index(axis=1)
        
        ### corr with time windows in past ###
#         wsize = 3
#         retesti = returns_test[(returns_test.index >= int(row['ID_DAY']-wsize)) & (returns_test.index <= int(row['ID_DAY']+wsize))]
#         retesti.index = range(0, 2*wsize+1)

#         max_cor = [0, 0, 0]
#         tab = {}
#         test = {}
#         for i in range(wsize, len(ret)-wsize):
#             reti = ret.iloc[i-wsize:i+wsize+1]
#             reti.index = range(0, 2*wsize+1)
#             cor = sum(reti.corrwith(retesti, axis=1).values)
#             tab[ret.iloc[i].name] = [cor, returns[returns.index == ret.iloc[i].name]['RET_'+str(int(row['ID_TARGET']))].values[0]]
#             t = [[cor], list(reti.corrwith(retesti, axis=1).values), [returns[returns.index == ret.iloc[i].name]['RET_'+str(int(row['ID_TARGET']))].values[0]]]
#             test[ret.iloc[i].name] = [item for sublist in t for item in sublist]
#         tab = pd.DataFrame(tab).T
#         tab_sort = tab.sort_values(by=0, ascending=False)
#         top = tab_sort.iloc[:11,]
#         bottom = tab_sort.iloc[-11:,]
#         pred1 = np.sign(top[top[1] > 0].count() - 5.1)[1]
#         pred2 = np.sign(5.1 - bottom[bottom[1] > 0].count())[1]
#         print(idx, pred1, pred2)
            
#         pd.DataFrame(test).T.to_excel('./dfexp.xlsx')
        ### --- ###
        
        r = row.sort_index(axis=0).iloc[2:,]
        diff_sign = (np.sign(ret)-np.sign(r).values) / 2
        diff_sign = diff_sign.abs().sum(axis=1).sort_values()
        diff_df = ret - r.values
        diff_df = diff_df.fillna(0)
        norm_df = diff_df.apply(np.linalg.norm, axis=1)
        norms = norm_df.sort_values()
        
        dfexp = {}
        for i in range(len(norms)):
            norme = round(norms.values[i], 3)
            dfexp[i] = [norme,
                        returns[ret.index == norms.index[i]]['RET_'+str(int(row['ID_TARGET']))].values[0],
                        1-norme,
                        diff_sign.values[diff_sign.index == norms.index[i]][0]]
        dfexp = pd.DataFrame(dfexp).T
        dfexp[4] = dfexp[1] * dfexp[2]
        dfexp[5] = np.where(dfexp[0] < dfexp[0].quantile(.5), dfexp[4], 0)
        dfexp[6] = np.where(dfexp[0] < dfexp[0].quantile(.25), dfexp[4], 0)
        dfexp[7] = np.where(dfexp[0] < dfexp[0].quantile(.15), dfexp[4], 0)
        dfexp[8] = np.where(dfexp[0] < dfexp[0].quantile(.10), dfexp[4], 0)
        dfexp[9] = np.where(dfexp[0] < dfexp[0].quantile(.05), dfexp[4], 0)
        dfexp[10] = np.where(dfexp[0] < dfexp[0].quantile(.02), dfexp[4], 0)
        dfexp[11] = np.where(dfexp[0] < dfexp[0].quantile(.01), dfexp[4], 0)
#         dfexp[12] = np.where((dfexp[0] < dfexp[0].quantile(.2)) & (dfexp[3] < 34), (0.4-dfexp[0])*(40-dfexp[3])*dfexp[1], 0)
#         dfexp[13] = np.where((dfexp[0] < dfexp[0].quantile(.1)) & (dfexp[3] < 34), (0.4-dfexp[0])*(40-dfexp[3])*dfexp[1], 0)
        pred[idx] = [dfexp[4].sum(), dfexp[5].sum(), dfexp[6].sum(),dfexp[7].sum(), dfexp[8].sum(), dfexp[9].sum(), dfexp[10].sum(), dfexp[11].sum()]
#         print(pred)
#         dfexp = pd.DataFrame(dfexp['RET_TARGET'].to_list(), index=dfexp.index)
        dfexp = dfexp.rename(columns = {0: 'norme',
                               1': 'pred',
                               2': '1-norme',
                               3': 'nb_diff_signe',
                               4': 'pred*(1-norme)',
                               5': 'pred*(1-norme) q0.5',
                               6': 'pred*(1-norme) q0.25',
                               7': 'pred*(1-norme) q0.15',
                               8': 'pred*(1-norme) q0.10',
                               9': 'pred*(1-norme) q0.05',
                               10: 'pred*(1-norme) q0.02',
                               11: 'pred*(1-norme) q0.01'})
        print(dfexp.columns)
        dfexp.to_excel('./dfexp.xlsx')


# pred = pd.Series(pred, name="RET_TARGET")
# pred = pd.DataFrame(pred).T
# pred = np.sign(pred)
# pred.to_excel('./dfexp.xlsx')

## Save the result before submission

In [69]:
pred.name = "RET_TARGET"
pred = pred.astype(int)
pred.to_csv('./benchmark_train_similar_rows.csv')